In [5]:
import pandas as pd
import numpy as np
from import_path import imp_path

Загрузка EXCEL

In [2]:
df = pd.read_excel(imp_path())

Тесты на корректность извлекаемых данных

In [3]:
#Проверка целостности данных
assert df['Ранее выданные лицензии'].count() == df['prev_lic'].count(), 'Ошибка в выгрузке предыдущих лицензий'
assert df['Сведения о переоформлении лицензии на пользование недрами'].count() == df['forw_lic'].count(), 'Ошибка в выгрузке будущих лицензий'
assert df['N'].count() == df['Координаты'].count(), 'Coords_extract_error'
assert df['Наименоваение недропользователя'].count() == df['Сведения о пользователе недр'].count(), 'Name_extract_error'
#assert df['INN'].count() == df['Сведения о пользователе недр'].count(), 'INN_extract_error' #не у всех есть ИНН в строке TODO: Придумать тест

In [4]:
print (df['prev_lic'].count(), df['Ранее выданные лицензии'].count())
print (df['forw_lic'].count(), df['Сведения о переоформлении лицензии на пользование недрами'].count())
print(df['N'].count(), df['Координаты'].count())

#Проверки соответствия данных
df['Ранее выданные лицензии'].compare(df['prev_lic'])
df['Ранее выданные лицензии'].compare(df['prev_lic']),df['Сведения о переоформлении лицензии на пользование недрами'].compare(df['forw_lic'])

#Проверки пустых значений
df[(df['forw_lic'].isna()) & (df['Сведения о переоформлении лицензии на пользование недрами'].notnull())]
df[(df['prev_lic'].isna()) & (df['Ранее выданные лицензии'].notnull())]
df[(df['Координаты'].isna()) & (df['N'].notnull())]

7910 7910
7904 7904
13285 13285


,Государственный регистрационный номер,Дата лицензии,СубъектРФ,Вид полезного ископаемого,Наименование участка недр,Сведения о пользователе недр,INN,Наименоваение недропользователя,prev_owner,Last,Year,Координаты,N,E,Сведения о переоформлении лицензии на пользование недрами,Ранее выданные лицензии,prev_lic,prev_date,forw_lic,forw_date
